### **<u>Item 68: Make pickle Reliable with copyreg</u>**
- copyreg를 사용해 pickle을 더 신뢰성 있게 만들자

#### <span style="color: yellowgreen">**_(1) pickle_**</span>
&emsp; - pickle은 python 객체를 직렬화하여 bin 형태로 저장하기 위해 제공되는 모듈 <br><br>
&emsp; - 버전이 서로 다른 상태에서 데이터를 load한 경우 신규 버전의 변수가 반영되지 않는다는 한계점 존재

In [1]:
# 피클링

import pickle

name = 'test'
content = 'hmm...'
date = {'year': 2023, 'month': 9, 'day':3}

with open('test_pickel.p', 'wb') as file:    # james.p 파일을 바이너리 쓰기 모드(wb)로 열기
    pickle.dump(name, file)
    pickle.dump(content, file)
    pickle.dump(date, file)

In [2]:
# 언피클링

with open('test_pickel.p', 'rb') as file:    # james.p 파일을 바이너리 쓰기 모드(wb)로 열기
    name = pickle.load(file)
    content = pickle.load(file)
    date = pickle.load(file)
    print(name, content, date)

test hmm... {'year': 2023, 'month': 9, 'day': 3}


&emsp; 1. **게임 상태 정의** <br><br>
&emsp;&emsp; - 게임의 상태를 관리하기 위해 GameState라는 클래스를 정의하고, 그 객체를 통해 현재 게임의 레벨과 남은 생명수를 추적

In [3]:
import pickle

class GameState:
    def __init__(self, level=0, lives=4):
        self.level = level
        self.lives = lives

# 게임 상태 저장
state = GameState()
state.level += 1
state.lives -= 1

&emsp; 2. **게임 상태 저장 및 호출** <br><br>
&emsp;&emsp; - pickle 모듈을 사용해서 GameState 객체를 파일에 저장 (직렬화) 하고, 나중에 이 파일을 다시 불러와 (역직렬화) 객체를 복원

In [4]:
state_path = 'game_state.bin'

# 객체 직렬화하여 파일에 저장
with open(state_path, 'wb') as f:
    pickle.dump(state, f)

#  파일에서 객체 역직렬화
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
    
print(state_after.__dict__)

{'level': 1, 'lives': 3}


&emsp; 3. **새로운 속성 추가** <br><br>
&emsp;&emsp; - 플레이어가 얻은 점수 points 속성을 새롭게 추가

In [5]:
class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 0  # New field

In [6]:
# 새로운 버전을 직렬화 하고 로드

state = GameState()
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'level': 0, 'lives': 4, 'points': 0}


&emsp; 4. **이전 게임 상태 로드 및 호환성 문제** <br><br>

In [9]:
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
    
print(state_after.__dict__)

{'level': 1, 'lives': 3}


#### <span style="color: yellowgreen">**_(2) copyreg_**</span>

&emsp; - copyreg모듈로 파이썬 객체를 직렬화할 함수를 등록하여 pickle의 동작을 제어하고 pickle을 더 신뢰할 수 있게 함 <br><br>

In [11]:
# 기본 인수가 있는 생성자를 사용
class GameState:
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points


#필요한 도우미 함수 만들기
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    return GameState(**kwargs)

In [12]:
import copyreg

# 내장 모듈 copyreg로 GameState 객체와 직렬화 함수를 등록
copyreg.pickle(GameState, pickle_game_state)

# 직렬화와 역직렬화는 이전과 동일하게 동작
state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'level': 0, 'lives': 4, 'points': 1000}


&emsp; 6. **다시 새로운 인자 추가** <br><br>

In [13]:
class GameState:
    def __init__(self, level=0, lives=4, points=0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic # New field

In [14]:
print('Before:', state.__dict__)
state_after = pickle.loads(serialized)
print('After: ', state_after.__dict__)

Before: {'level': 0, 'lives': 4, 'points': 1000}
After:  {'level': 0, 'lives': 4, 'points': 1000, 'magic': 5}


#### <span style="color: yellowgreen">**_Summary_**</span>

&emsp; &#9312; pickle 내장 모듈은 신뢰할 수 있는 프로그램 간의 객체를 직렬화 하고 역직렬화하는 데에만 유용함<br><br>
&emsp; &#9313; 이전에 피클된 객체를 역직렬화 하는 작업은 관련 클래스가 시간이 지남에 따라 변경된 경우 중단될 수 있음 <br><br>
&emsp; &#9314; 따라서 copyreg 모듈을 함께 사용하여 직렬화된 객체의 이전 버전과의 호환성을 보장해야함